# SQL query generation using the OPEN AI API 

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a Model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [22]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [6]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [8]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['employees', 'salary', 'studies'],
        'definition': ['Employee information, name...',
                       'Salary details for each year',
                       'Educational studies, name of the institution, type of studies, level']}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees                      Employee information, name...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [10]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [14]:
print(text_tables)

employees: Employee information, name...
salary: Salary details for each year
studies: Educational studies, name of the institution, type of studies, level


In [16]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [18]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="Return The name of the best paid employee")

In [24]:
print(return_OAI(pqt1))

```json
{
    "tables": ["employees", "salary"]
}
```


In [26]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="Return the Education Institution with a higher average salary")

In [28]:
print(return_OAI(pqt3))

```json
{
    "tables": {
        "employees": "Employee information",
        "salary": "Salary details for each year",
        "studies": "Educational studies"
    }
}
```


# Conclusions
It is clear that GPT-3.5-Turbo is a model entirely capable of deciding which tables should be used in creating an SQL query.

In a more complex system, we may need to refine the definition and conduct several tests before finalizing them.
